## DistilBert vs. RuBert on test data

### Сетап

In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from datasets import load_dataset

In [2]:
DATASET_NAME = "kuznetsoffandrey/sberquad"
dataset_test = load_dataset(DATASET_NAME, split="test")

import random

dataset_test = list(dataset_test)
random.shuffle(dataset_test)

### Проверка работы модели DistilBert на части тестовых данных

In [3]:
MODEL_SAVE_DIR = './distilbert-v2'

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_SAVE_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_DIR)

In [4]:
for example in dataset_test[:5]:
    question = example['question']
    context = example['context']
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    answer_tokens = inputs['input_ids'][0][start_idx: end_idx + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    print(f"Контекст: {context}")
    print(f"Вопрос: {question}")
    print(f"Ответ: {answer}\n")

Контекст: Пестики многочисленные, сидячие или на коротких ножках, вросшие в цветоложе, расположенные по спирали[52] на дне гипантия. Завязь волосистая, одногнёздная, свободная, сидячая или на ножке[53], с одной сидячей семяпочкой. Семяпочка с одним покровом[10]. Стилодии свободные или соединены в плотную колонку (при этом многочисленные рыльца, соединённые в пучок, создают удобную площадку для посадки насекомых[54], расположены на верхушке плодолистика, почти целиком заключены в гипантий или выступают из него, иногда превышают внутренний круг тычинок, голые или опушённые. Стилодий при созревании плода может оставаться на завязи, но быстро увядает и теряет свою форму, как у шиповника морщинистого[10]. Рыльца головчатые, голые или густоволосистые[9].
Вопрос: Какого строение стилодии шиповника?
Ответ: морщинистого

Контекст: Домашние куры и петухи являются значимым элементом русской культуры, используются в фольклоре, народных сказках и литературных произведениях, где являются в том числе

### Проверка работы модели RuBert на части тестовых данных

In [5]:
MODEL_SAVE_DIR = './rubert-v3'

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_SAVE_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_DIR)

In [6]:
for example in dataset_test[:5]:
    question = example['question']
    context = example['context']
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    answer_tokens = inputs['input_ids'][0][start_idx: end_idx + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    print(f"Контекст: {context}")
    print(f"Вопрос: {question}")
    print(f"Ответ: {answer}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Контекст: Пестики многочисленные, сидячие или на коротких ножках, вросшие в цветоложе, расположенные по спирали[52] на дне гипантия. Завязь волосистая, одногнёздная, свободная, сидячая или на ножке[53], с одной сидячей семяпочкой. Семяпочка с одним покровом[10]. Стилодии свободные или соединены в плотную колонку (при этом многочисленные рыльца, соединённые в пучок, создают удобную площадку для посадки насекомых[54], расположены на верхушке плодолистика, почти целиком заключены в гипантий или выступают из него, иногда превышают внутренний круг тычинок, голые или опушённые. Стилодий при созревании плода может оставаться на завязи, но быстро увядает и теряет свою форму, как у шиповника морщинистого[10]. Рыльца головчатые, голые или густоволосистые[9].
Вопрос: Какого строение стилодии шиповника?
Ответ: свободные или соединены в плотную колонку

Контекст: Домашние куры и петухи являются значимым элементом русской культуры, используются в фольклоре, народных сказках и литературных произведен

### Анализ

Мы видим, что натренированный RuBert показывает себя немного лучше чем DistilBert. Хотя и он не идеален.